### 1. WebSearchRetriever

Code: https://api.python.langchain.com/en/latest/_modules/langchain/retrievers/web_research.html#WebResearchRetriever

[Requirements]
- llm
- search_api
- vectorstore

In [59]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.chat_models import BedrockChat
import boto3
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import RetrievalQAWithSourcesChain

import os, json
with open("/home/ubuntu/config.json") as f:
    config = json.loads(f.read())
os.environ["COHERE_API_KEY"] = config["cohere_api_key"]

import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)


In [60]:
# WebResearchRetriever's code makes it compulsory to use GoogleSearchAPIWrapper

! pip install google-api-python-client>=2.100.0 --quiet

from langchain_community.utilities import GoogleSearchAPIWrapper
os.environ["GOOGLE_API_KEY"] = config["google_api_key"]
os.environ["GOOGLE_CSE_ID"] = config["google_cse_id"]

# from langchain_community.utilities import GoogleSerperAPIWrapper
# os.environ["SERPER_API_KEY"] = config["serper_api_key"]

In [61]:
# install for AsyncHtml, HTML to text
! pip install html2text --quiet

In [62]:
vectorstore = Chroma(embedding_function=CohereEmbeddings())

llm = BedrockChat(
    # model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={"temperature": 0.0, "max_tokens":512}
)

search = GoogleSearchAPIWrapper()

web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)

In [63]:
%%time

question = "How do LLM Powered Autonomous Agents work ?"
response = qa_chain.invoke({"question": question})

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'How do LLM Powered Autonomous Agents work ?', 'text': ['1. What is the mechanism behind Large Language Model-based Autonomous Agents?\n', '2. How do AI systems powered by LLMs operate autonomously?\n', '3. Can you explain the working principle of LLM-driven Autonomous Agents?']}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What is the mechanism behind Large Language Model-based Autonomous Agents?\n', '2. How do AI systems powered by LLMs operate autonomously?\n', '3. Can you explain the working principle of LLM-driven Autonomous Agents?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': 'A Survey on Large Language Model based Autonomous 

CPU times: user 1.51 s, sys: 526 ms, total: 2.04 s
Wall time: 14.7 s


In [65]:
print(response["answer"])

LLM Powered Autonomous Agents work by combining a large language model (LLM) with several key components:

1. Planning:
   - Task Decomposition: The agent breaks down complex tasks into smaller subgoals, enabling efficient handling of large tasks.
   - Self-Reflection: The agent can self-critique and learn from past actions, refining its approach for future steps.

2. Memory:
   - Short-term Memory: Utilizing in-context learning to remember recent information.
   - Long-term Memory: Leveraging external vector stores and fast retrieval to retain and recall large amounts of information over extended periods.

3. Tool Use: 
   - The agent can call external APIs to access information missing from the model weights, such as current data, code execution capabilities, and proprietary information sources.

The LLM acts as the "brain" of the agent, complemented by these components for planning, memory management, and tool utilization. This allows the agent to handle complex tasks, learn from ex

In [66]:
print(response["sources"])

https://lilianweng.github.io/posts/2023-06-23-agent/


### 2. Custom approach

[Steps]
- Search: query -> url
    - GoogleSearchAPIWrapper
    - GoogelSerperAPIWrapper
- Loader: url -> html
    - AsyncHtmlLoader
    - AsyncChromiumLoader
- Transformer: html -> formatted text
    - HTML2Text
    - BeautifulSoup

#### search

In [1]:
import os, json
with open("/home/ubuntu/config.json") as f:
    config = json.loads(f.read())

In [2]:
from langchain_community.utilities import GoogleSearchAPIWrapper
os.environ["GOOGLE_API_KEY"] = config["google_api_key"]
os.environ["GOOGLE_CSE_ID"] = config["google_cse_id"]

search = GoogleSearchAPIWrapper()
question = "How to build autonomous web research agent ?"
response = search.results(query=question, num_results=3)
response

[{'title': 'How I Built an Autonomous AI Agent for Online Research | by Assaf ...',
  'link': 'https://betterprogramming.pub/how-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c',
  'snippet': 'Jul 24, 2023 ... Another issue with AutoGPT is that it works synchronously. The main idea of it is to create a list of tasks and then execute them one by one. So\xa0...'},
 {'title': 'Automating Web Research',
  'link': 'https://blog.langchain.dev/automating-web-research/',
  'snippet': 'Jul 26, 2023 ... What started as an attempt to build an autonomous web research agent, evolved into a fairly simple / efficient and customizable retriever.'},
 {'title': 'Autonomous AI Agents | Taskade Help Center',
  'link': 'https://help.taskade.com/en/articles/8958458-autonomous-ai-agents',
  'snippet': 'Conduct in-depth web research automatically. Generate creative writing pieces or articles. Answer complex questions on various topics. Assist with coding and\xa0...'}]

In [3]:
from langchain_community.utilities import GoogleSerperAPIWrapper
os.environ["SERPER_API_KEY"] = config["serper_api_key"]

# search = GoogleSerperAPIWrapper(type="news", k=3)
search = GoogleSerperAPIWrapper(type="search", k=3)
question = "How to build autonomous web research agent ?"
response = search.results(query=question)
response

{'searchParameters': {'q': 'How to build autonomous web research agent ?',
  'gl': 'us',
  'hl': 'en',
  'type': 'search',
  'num': 3,
  'engine': 'google'},
 'organic': [{'title': 'How I Built an Autonomous AI Agent for Online Research',
   'link': 'https://betterprogramming.pub/how-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c',
   'snippet': 'Generate an outline of research questions that form an objective opinion on any given task. For each research question, trigger a crawler agent ...',
   'date': 'Jul 24, 2023',
   'position': 1},
  {'title': 'Automating Web Research - LangChain Blog',
   'link': 'https://blog.langchain.dev/automating-web-research/',
   'snippet': 'What started as an attempt to build an autonomous web research agent, evolved into a fairly simple / efficient and customizable retriever.',
   'date': 'Jul 26, 2023',
   'position': 2},
  {'title': 'Build an Autonomous Researcher - AI Jason',
   'link': 'https://www.ai-jason.com/learning-ai/how-to-bu

In [4]:
from pprint import pprint

pprint(response)

{'organic': [{'date': 'Jul 24, 2023',
              'link': 'https://betterprogramming.pub/how-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c',
              'position': 1,
              'snippet': 'Generate an outline of research questions that form '
                         'an objective opinion on any given task. For each '
                         'research question, trigger a crawler agent ...',
              'title': 'How I Built an Autonomous AI Agent for Online '
                       'Research'},
             {'date': 'Jul 26, 2023',
              'link': 'https://blog.langchain.dev/automating-web-research/',
              'position': 2,
              'snippet': 'What started as an attempt to build an autonomous '
                         'web research agent, evolved into a fairly simple / '
                         'efficient and customizable retriever.',
              'title': 'Automating Web Research - LangChain Blog'},
             {'attributes': {'Missin

In [5]:
urls = [x["link"] for x in response["organic"]]
urls

['https://betterprogramming.pub/how-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c',
 'https://blog.langchain.dev/automating-web-research/',
 'https://www.ai-jason.com/learning-ai/how-to-build-an-autonomous-researcher-using-gpt-a-step-by-step-guide']

#### loader

In [19]:
%%time
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ['https://betterprogramming.pub/how-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c',
 'https://blog.langchain.dev/automating-web-research/',
 'https://www.ai-jason.com/learning-ai/how-to-build-an-autonomous-researcher-using-gpt-a-step-by-step-guide']

loader = AsyncHtmlLoader(urls)
docs = loader.load()

docs

Fetching pages: 100%|##########| 3/3 [00:00<00:00,  5.25it/s]


CPU times: user 145 ms, sys: 4.86 ms, total: 150 ms
Wall time: 1.47 s


[Document(page_content='<!doctype html><html lang="en"><head><title data-rh="true">How I Built an Autonomous AI Agent for Online Research | by Assaf Elovic | Better Programming</title><meta data-rh="true" charset="utf-8"/><meta data-rh="true" name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1"/><meta data-rh="true" name="theme-color" content="#000000"/><meta data-rh="true" name="twitter:app:name:iphone" content="Medium"/><meta data-rh="true" name="twitter:app:id:iphone" content="828256236"/><meta data-rh="true" property="al:ios:app_name" content="Medium"/><meta data-rh="true" property="al:ios:app_store_id" content="828256236"/><meta data-rh="true" property="al:android:package" content="com.medium.reader"/><meta data-rh="true" property="fb:app_id" content="542599432471018"/><meta data-rh="true" property="og:site_name" content="Medium"/><meta data-rh="true" property="og:type" content="article"/><meta data-rh="true" property="article:published_time

In [12]:
# %%time
# ! pip install playwright --quiet
# # getting runtime error - need to fix it

# from langchain_community.document_loaders import AsyncChromiumLoader

# urls = ['https://betterprogramming.pub/how-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c',
#  'https://blog.langchain.dev/automating-web-research/',
#  'https://www.ai-jason.com/learning-ai/how-to-build-an-autonomous-researcher-using-gpt-a-step-by-step-guide']

# loader = AsyncChromiumLoader(urls)
# docs = loader.load()

# docs

#### transformer

In [13]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()

docs_transformed = html2text.transform_documents(docs)
print(docs_transformed[0].page_content[:500])

Open in app

Sign up

Sign in

Write

Sign up

Sign in

Member-only story

# How I Built an Autonomous AI Agent for Online Research

## GPT Researcher: the future of comprehensive online research

Assaf Elovic

·

Follow

Published in

Better Programming

·

6 min read

·

Jul 24, 2023

\--

10

Share

After AutoGPT was published, I immediately took it for a spin. The first use
case that came to mind was autonomous online research. Forming objective
conclusions for manual research tasks can take


In [21]:
# from langchain_community.document_transformers import BeautifulSoupTransformer

# bstransformer = BeautifulSoupTransformer()

# docs_transformed = bstransformer.transform_documents(docs, tags_to_extract=["span"])
# print(docs_transformed[0].page_content[:500])

Sign up (https://medium.com/m/signin?operation=register&redirect=https%3A%2F%2Fbetterprogramming.pub%2Fhow-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c&source=post_page---two_column_layout_nav-----------------------global_nav-----------) Sign in (https://medium.com/m/signin?operation=login&redirect=https%3A%2F%2Fbetterprogramming.pub%2Fhow-i-built-an-autonomous-ai-agent-for-online-research-93435a97c6c&source=post_page---two_column_layout_nav-----------------------global_nav----


#### combining all - simple version

In [5]:
import os, json
with open("/home/ubuntu/config.json") as f:
    config = json.loads(f.read())
os.environ["SERPER_API_KEY"] = config["serper_api_key"]

In [10]:
from typing import List
from langchain_core.documents import Document
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer

def simple_web_retriever(
    query: str, 
    top_k: int = 3
) -> List[Document]:
    response = GoogleSerperAPIWrapper(type="search", k=top_k).results(query=query)
    urls = [x["link"] for x in response["organic"]]
    docs = AsyncHtmlLoader(urls).load()
    docs_transformed = Html2TextTransformer().transform_documents(docs)
    return docs_transformed

In [11]:
%%time

question = "How to build autonomous web research agent ?"
docs = simple_web_retriever(query=question, top_k=3)

Fetching pages: 100%|##########| 3/3 [00:00<00:00,  6.04it/s]


CPU times: user 379 ms, sys: 9.18 ms, total: 388 ms
Wall time: 2.01 s


In [22]:
for i, doc in enumerate(docs):
    print(f"doc_{i}")
    print(f"#"*25)
    print(f"page_content: \n{doc.page_content}")
    print(f"metadata: \n{doc.metadata}")
    print(f"#"*40)

doc_0
#########################
page_content: 
Skip to content

  * By LangChain
  * Release Notes
  * Case Studies
  * LangChain
  * GitHub
  * Docs

Sign in Subscribe

# Automating Web Research

4 min read Jul 26, 2023

## Key Links

  * Web Researcher Repo
  * New LangChain Retriever and Documentation
  * Hosted Streamlit App

## Motivation

Web research is one of the killer LLM applications: Greg Kamradt highlighted
it as one of his top desired AI tools and OSS repos like gpt-researcher are
growing in popularity. We decided to take a stab at it, initially setting out
like many others to build a web research agent. But, we landed somewhere
different: a fairly simple retriever proved to be effective and easily
configurable (e.g., to run in private mode as popularized by projects like
PrivateGPT) . In this blog we talk about our exploration and thought process,
how we built it, and the next steps.

## Exploration

Abovementioned projects like gpt-researcher and AI search engines
(perp

#### combining all - better version

[steps]
1. search
2. load
3. transform
4. split -> store in vectorstores
5. search in vectostore for relevant splits
6. return unique docs

In [18]:
import os, json
with open("/home/ubuntu/config.json") as f:
    config = json.loads(f.read())
os.environ["COHERE_API_KEY"] = config["cohere_api_key"]
os.environ["SERPER_API_KEY"] = config["serper_api_key"]

In [68]:
from typing import List
from langchain_core.documents import Document
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

def better_web_retriever(
    query: str, 
    top_k: int = 3
) -> List[Document]:
    response = GoogleSerperAPIWrapper(type="search", k=top_k).results(query=query)
    
    urls = [x["link"] for x in response["organic"]]
    docs = AsyncHtmlLoader(urls).load()
    
    docs_transformed = Html2TextTransformer().transform_documents(docs)
   
    vectorstore = Chroma(embedding_function=CohereEmbeddings())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
    docs_splitted = text_splitter.split_documents(docs_transformed)
    vectorstore.add_documents(docs_splitted)
    
    relevant_docs = vectorstore.similarity_search(query)
    
    unique_relevant_dict = {
        (doc.page_content, tuple(sorted(doc.metadata.items()))): doc for doc in relevant_docs
    }
    unique_relevant_docs = list(unique_relevant_dict.values())
    return unique_relevant_docs

In [69]:
%%time

question = "How to build autonomous web research agent ?"
docs = better_web_retriever(query=question, top_k=3)

Fetching pages: 100%|##########| 3/3 [00:00<00:00,  4.67it/s]


CPU times: user 366 ms, sys: 9.68 ms, total: 376 ms
Wall time: 8.5 s


In [70]:
for i, doc in enumerate(docs):
    print(f"doc_{i}")
    print(f"-"*25)
    print(f"page_content: \n{doc.page_content}")
    print(f"metadata: \n{doc.metadata}")
    print(f"-"*40)

doc_0
-------------------------
page_content: 
## Conclusion

What started as an attempt to build an autonomous web research agent, evolved
into a fairly simple / efficient and customizable retriever. Still, this was
just a first step. This project could benefit from adding in many agentic
properties, such as:

  * Asking an LLM if more information is needed after the initial search
  * Using multiple "write" and "revision" agents to construct the final answer

If any of those additions sound interesting, please open a PR against the base
repo and we'll work with you to get them in!

While hosted AI search from large models like Bard or Perplexity.ai are
extremely performant, smaller lightweight tools for web research also have
important merits such as privacy (e.g., the ability to run locally on your
laptop without sharing any data externally), configurability (e.g., the
ability to select the specific open source components to use), and
observability (e.g., peer into what is happening

#### converting above fns to retrievers and apply qa-chain

In [90]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

class SimpleWebRetriever(BaseRetriever):
    """Simple web retriever"""
    top_k: int=3

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        try:
            response = GoogleSerperAPIWrapper(type="search", k=self.top_k).results(query=query)
            urls = [x["link"] for x in response["organic"]]
            docs = AsyncHtmlLoader(urls).load()
            docs_transformed = Html2TextTransformer().transform_documents(docs)
            return docs_transformed
        except Exception as e:
            print(f"Exception received: \n{e}")
        return None

class BetterWebRetriever(BaseRetriever):
    "better web retriever"
    top_k: int=3

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        try:
            response = GoogleSerperAPIWrapper(type="search", k=self.top_k).results(query=query)
    
            urls = [x["link"] for x in response["organic"]]
            docs = AsyncHtmlLoader(urls).load()
            
            docs_transformed = Html2TextTransformer().transform_documents(docs)
        
            vectorstore = Chroma(embedding_function=CohereEmbeddings())
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
            docs_splitted = text_splitter.split_documents(docs_transformed)
            vectorstore.add_documents(docs_splitted)
            
            relevant_docs = vectorstore.similarity_search(query)
            
            unique_relevant_dict = {
                (doc.page_content, tuple(sorted(doc.metadata.items()))): doc for doc in relevant_docs
            }
            unique_relevant_docs = list(unique_relevant_dict.values())
            return unique_relevant_docs
        except Exception as e:
            print(f"Exception received: \n{e}")
        return None

In [91]:
question = 'How to build autonomous web research agent ?'

simple_web_retriever = SimpleWebRetriever(top_k=10)

docs = simple_web_retriever.get_relevant_documents(question)
print(len(docs))
docs

Fetching pages: 100%|##########| 10/10 [00:01<00:00,  6.96it/s]


10


[Document(page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nMember-only story\n\n# How I Built an Autonomous AI Agent for Online Research\n\n## GPT Researcher: the future of comprehensive online research\n\nAssaf Elovic\n\n·\n\nFollow\n\nPublished in\n\nBetter Programming\n\n·\n\n6 min read\n\n·\n\nJul 24, 2023\n\n\\--\n\n10\n\nShare\n\nAfter AutoGPT was published, I immediately took it for a spin. The first use\ncase that came to mind was autonomous online research. Forming objective\nconclusions for manual research tasks can take time, sometimes weeks, to find\nthe right resources and information. Seeing how well AutoGPT created tasks and\nexecuted them got me thinking about the great potential of using AI to conduct\ncomprehensive research and what it meant for the future of online research.\n\nBut the problem with AutoGPT was that it usually ran into never-ending loops,\nrequired human interference for almost every step, constantly lost track of\nits

In [92]:
question = 'How to build autonomous web research agent ?'

better_web_retriever = BetterWebRetriever(top_k=10)

docs = better_web_retriever.get_relevant_documents(question)
print(len(docs))
docs

Fetching pages: 100%|##########| 9/9 [00:01<00:00,  5.68it/s]


2


[Document(page_content='* Skip to main content\n  * Skip to secondary menu\n  * Skip to primary sidebar\n  * Skip to footer\n\nGeeky Gadgets\n\nThe Latest Technology News\n\n  * Home\n  * Apple\n  * Android\n  * Deals\n  * Gadgets\n  * Technology\n  * Hardware\n  * Gaming\n  * Guides\n  * Auto News\n\n  \n\n  \n\n# How to build an autonomous AI research agent running 24/7\n\n2:35 pm September 22, 2023 By Julian Horsey\n\nIf you would like to build your very own **autonomous AI research agent** that\nis capable of**running 24-hours** a day seven days a week scouring the web for\nanything you ask it. You might be interested in a new tutorial by Michael\nBorman who has kindly uploaded the code to GitHub for you to download and\ntweak to your exact requirements.\n\nBuilding a research agent using**LangChain** and integrating it into a product\nis a complex yet rewarding process. This process involves several steps,\nincluding building a tool for search and scraping, creating a Lang chain\n

In [94]:
pprint(docs[0].metadata)

{'description': 'Learn how to build an autonomous AI research agent using '
                'LangChain allowing you to can doubt research 24-hour is a day '
                'seven days a week.',
 'language': 'en-US',
 'source': 'https://www.geeky-gadgets.com/build-an-autonomous-ai-research-agent/',
 'title': 'How to build an autonomous AI research agent running 24/7 - Geeky '
          'Gadgets'}


In [98]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_community.chat_models import BedrockChat
import boto3

llm = BedrockChat(
    # model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={"temperature": 0.0, "max_tokens":512}
)

# simple_web_retriever = SimpleWebRetriever(top_k=10)
better_web_retriever = BetterWebRetriever(top_k=10)

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
    # retriever=simple_web_retriever,
    retriever=better_web_retriever,
)

In [99]:
%%time

question = "How do LLM Powered Autonomous Agents work ?"
response = qa_chain.invoke({"question": question})

Fetching pages: 100%|##########| 10/10 [00:00<00:00, 10.16it/s]


CPU times: user 2.6 s, sys: 75.9 ms, total: 2.67 s
Wall time: 15.5 s


In [100]:
pprint(response)

{'answer': 'LLM-Powered Autonomous Agents work by using a large language model '
           '(LLM) as the core controller or "brain" of the agent system, '
           'complemented by several key components:\n'
           '\n'
           '1. Planning:\n'
           '   - Task Decomposition: The agent breaks down complex tasks into '
           'smaller, manageable subgoals.\n'
           '   - Reflection and Refinement: The agent can self-reflect on past '
           'actions, learn from mistakes, and refine its approach for future '
           'steps.\n'
           '\n'
           '2. Memory:\n'
           '   - Short-term Memory: Utilizing in-context learning and prompt '
           "engineering to leverage the model's short-term memory.\n"
           '   - Long-term Memory: Integrating external vector stores or '
           'databases to retain and recall information over extended periods.\n'
           '\n'
           '3. Tool Use:\n'
           '   - The agent can call external AP

In [101]:
print(response["answer"])

LLM-Powered Autonomous Agents work by using a large language model (LLM) as the core controller or "brain" of the agent system, complemented by several key components:

1. Planning:
   - Task Decomposition: The agent breaks down complex tasks into smaller, manageable subgoals.
   - Reflection and Refinement: The agent can self-reflect on past actions, learn from mistakes, and refine its approach for future steps.

2. Memory:
   - Short-term Memory: Utilizing in-context learning and prompt engineering to leverage the model's short-term memory.
   - Long-term Memory: Integrating external vector stores or databases to retain and recall information over extended periods.

3. Tool Use:
   - The agent can call external APIs, execute code, access proprietary information sources, and leverage other tools to augment its capabilities beyond what is encoded in the LLM's weights.

The LLM acts as the central controller, understanding natural language instructions, decomposing tasks, making decisio

In [102]:
print(response["sources"])